In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
rawDataset = pd.read_csv("../input/letterrecognition-using-svm/letter-recognition.csv")
dddata = rawDataset['letter']
rawDataset = rawDataset.drop("letter", axis = 1)
rawDataset1 = rawDataset.join(dddata)
rawDataset = rawDataset1.sample(frac=0.0015)
# 决策分割点
split = 16
dataset = rawDataset.copy()
dataset

In [ ]:
for column in ["letter"]:
    oneHot = pd.get_dummies(dataset[column], prefix=column)
    dataset = dataset.join(oneHot)
    dataset = dataset.drop(column, axis = 1)
dataset

In [ ]:
dataset = (dataset - dataset.min()) / (dataset.max() - dataset.min())
dataset

In [ ]:
def cover(dataset, baseList, isCopy = True):
    "决策信息系统到决策形式背景"
    "dataset: 决策信息系统;baseList:基准列表;isCopy:是否复制副本:return:决策形式背景"
    if isCopy:
        dataset = dataset.copy()
    for i in range(dataset.shape[1]-1):
        base = baseList[i]
        big = dataset.iloc[:,i] >= base
        small = dataset.iloc[:,i] < base
        dataset.iloc[big,i] = 1
        dataset.iloc[small,i] = 0
    return dataset

def contain(array1, array2):
    "辅助函数"
    for x in array2:
        if x not in array1:
            return False
    return True

def isCoordinated(row, attrdataset, decidataset):
    "判断当前行是否粒协调"
    "row:行;attrdataset:条件属性形式背景;decidataset:决策属性形式背景;return:是否粒协调"
    xaa = []
    xdd = []
#     print(attrdataset)
#     print(decidataset)
    for index in attrdataset.index:
        isAddxaa = True
        for column in attrdataset:
            if attrdataset.loc[index, column] == 0 and attrdataset.loc[row, column] == 1:
                isAddxaa = False
        if isAddxaa:
            xaa.append(index)
#     print(xaa)
        
    for index in decidataset.index:
        isAddxdd = True
        for column in decidataset:
            if decidataset.loc[index, column] == 0 and decidataset.loc[row, column] == 1:
                isAddxdd = False
        if isAddxdd:
            xdd.append(index)
#     print(xdd)
    if contain(xdd, xaa):
        return True
    return False

def coordinatedRate(dataset, split):
    "粒协调率"
    "dataset:决策形式背景;split:int类型的分割，在这之前是条件属性（不包括这个），之后是决策属性，使用方法参考list split"
    trueNum = 0
    falseNum = 0
    isTrue = []
    for index in dataset.index:
        if isCoordinated(index, dataset.iloc[:, :split], dataset.iloc[:, split:]):
            trueNum += 1
            isTrue.append(True)
        else:
            falseNum += 1
            isTrue.append(False)
    rate = trueNum / (trueNum + falseNum)
    return rate,isTrue


In [ ]:
dataProcessed = dataset.copy()
dataProcessed

In [ ]:
# 初始化初始解
baseList = np.ones(dataProcessed.shape[1], int)*0.5
maxCoa = 0
for a in range(5):
    # 迭代次数
    maxCob = 0
    for l in range(split):
        # 对一个维度进行搜索
        # 这个维度可能取值的列表
        listb = list(dataProcessed.iloc[:,l])
        #列表去重
        listb = list(set(listb))
        listb.sort()
        maxCo = 0
        maxBase = 0
        for b in listb:
            # 对这个维度可能取值进行循环取值
            baseList[l] = b
            coRate, isTrue = coordinatedRate(cover(dataProcessed, baseList), split)
            if coRate > maxCo:
                maxCo = coRate
                maxBase = b
            print("迭代:",a+1,"维度:",l,"取值:",b,"协调率:",coRate,"当前解:",baseList)
        baseList[l] = maxBase
        print("*** 迭代:",a+1,"维度:",l,"协调率:",maxCo,"最佳解:",baseList)
        maxCob = maxCo
    if maxCob == maxCoa:
        break
    else:
        maxCoa = maxCob
print("*** 协调率:",maxCoa,"最佳解:",baseList)

In [ ]:
bestdata = cover(dataProcessed, baseList)
bestdata

In [ ]:
coRate, isTrue = coordinatedRate(bestdata, split)
coordinatedDataset = bestdata[isTrue]
coordinatedDataset

In [ ]:
u = coordinatedDataset.index.values
a = coordinatedDataset.columns.values[:split]
d = coordinatedDataset.columns.values[split:]
i = coordinatedDataset
print(u,a,d,i)

In [ ]:
# 构造辨识矩阵
discernMat = []
for xa in range(0, len(u)):
#     print("xa",xa)
    dRow = []
    for xb in range(0, len(u)):
#         print("xb",xb)
        dBlock = []
        for attr in range(0, len(a)):
#             print("attr",attr)
            if i.iloc[xa,attr] == 1 and i.iloc[xb,attr] != 1 and list(i.iloc[xb,split:]) != list(i.iloc[xa,split:]):
                dBlock.append(a[attr])
        dRow.append(dBlock)
    discernMat.append(dRow)
discernMat

In [ ]:
dMat = discernMat
dList = []
for y in dMat:
    for x in y:
        if x not in dList:
            dList.append(x)
dList.remove([])
print(dList)

In [ ]:
removeList = []
for aList in dList:
    for aListCompare in dList:
        if aListCompare != aList:
            if contain(aListCompare,aList):
                if aListCompare not in removeList:
                    removeList.append(aListCompare)
for q in removeList:
    dList.remove(q)
print(dList)
print("粒约简：")
granularList = []
import itertools
for item in itertools.product(*dList):
    granularList = item
    print(item)
list(granularList)

In [ ]:
removeAttr = a.copy()
removeAttr = np.setdiff1d(removeAttr, granularList)
removeAttr

In [ ]:
reductedData = rawDataset1.copy()
reductedData = reductedData.drop(removeAttr, axis=1)
reductedData

In [ ]:
from sklearn import tree
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split

In [ ]:
# 约简前
X, y = rawDataset1.iloc[:,:split], rawDataset1.iloc[:,split:]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)
clf = tree.DecisionTreeClassifier()
clf = clf.fit(X_train, y_train)
print(tree.export_text(clf, feature_names=list(a)))
pred = clf.predict(X_test)
confusion_matrix(y_test, pred)

In [ ]:
# 约简后
X, y = reductedData.iloc[:,:len(granularList)], reductedData.iloc[:,len(granularList):]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)
clf = tree.DecisionTreeClassifier()
clf = clf.fit(X_train, y_train)
print(tree.export_text(clf, feature_names=granularList))
pred = clf.predict(X_test)
confusion_matrix(y_test, pred)